In [2]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.animation import FuncAnimation

import os
import glob
import pandas as pd
import json
from dotenv import load_dotenv
from functools import partial

from utils.db_tools import get_db, metrics_grid, plot_grid, get_data, filter_df
from classify import classify_trajectories

In [3]:
model = "bruss"
run_id = "ball_big"
load_dotenv()
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUT_DIR")
output_dir = os.path.join(output_dir, model, run_id)
os.makedirs(output_dir, exist_ok=True)
df = pd.read_csv(os.path.join(output_dir, "classification_metrics_02.csv"))
df_class = classify_trajectories(
    df
)

In [4]:
df_class.category.value_counts()

category
SS     3007
OSC    2128
INT    1676
DSS     387
Name: count, dtype: int64

In [5]:
from collections import defaultdict

Du_values = [1.0, 2.0, 3.0]
D_mult_values = [4, 11, 18]
A_values = [0.1, 0.5, 1, 2, 5, 10]
B_mult_values = [1.25, 1.75, 2, 2.5, 3, 4, 5]

# Initialize dictionaries to store distributions for each parameter
category_distribution = {
    "A": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
    "B_mult": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
    "Du": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
    "D_mult": defaultdict(lambda: {"steady_state": 0, "interesting_behavior": 0, "divergent_or_unknown": 0}),
}

# Iterate over all parameter combinations
for Du in Du_values:
    for D_mult in D_mult_values:
        Dv = Du * D_mult
        df_filt = df[(df["Du"] == Du) & (df["Dv"] == Dv)]

        # Update category distributions
        for A in sorted(A_values):
            for B_mult in B_mult_values:
                B = A * B_mult
                row = df_class[(df_class["A"] == A) & (df_class["B"] == B)]
                if not row.empty:
                    category = row["category"].values[0]
                    category_distribution["A"][A][category] += 1
                    category_distribution["B_mult"][B_mult][category] += 1
                    category_distribution["Du"][Du][category] += 1
                    category_distribution["D_mult"][D_mult][category] += 1
                    
# Print distributions for each parameter
def print_distributions(param_name, distribution):
    print(f"Distributions for {param_name}:")
    for param_value, counts in sorted(distribution.items()):
        total = sum(counts.values())
        print(f"  {param_value}: {counts} (total={total})")
    print("")

print_distributions("A", category_distribution["A"])
print_distributions("B_mult", category_distribution["B_mult"])
print_distributions("Du", category_distribution["Du"])
print_distributions("D_mult", category_distribution["D_mult"])


Distributions for A:

Distributions for B_mult:

Distributions for Du:

Distributions for D_mult:

